In [1]:
#########
### importing bla bla bla ####
import pandas as pd
import numpy as np
import seaborn as sb
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)

from sklearn.cross_validation import train_test_split
from pandas import Series, DataFrame
from pylab import rcParams


from sklearn.metrics import classification_report,accuracy_score
sb.set(style="white")
sb.set(style="whitegrid", color_codes=True)
import itertools
from sklearn import tree
import graphviz
from sklearn.model_selection import GridSearchCV,StratifiedKFold, LeaveOneOut
from sklearn.datasets import make_regression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE

import warnings; warnings.simplefilter('ignore')






### done with the imports ###



C:\Users\projna.paromita\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
### reading data and printing the data ###
data = pd.read_csv('classical_feature.csv',names=['TruncatedID','ID','Mean','SD','AUC','Peak','MaxIndex','Halfpk','Skew','HHMean','Fat','Carb','Protein'
])
# data = data.dropna()
print(data.shape)
print(list(data.columns))


### done with reading data and printing the data ###

(171, 13)
['TruncatedID', 'ID', 'Mean', 'SD', 'AUC', 'Peak', 'MaxIndex', 'Halfpk', 'Skew', 'HHMean', 'Fat', 'Carb', 'Protein']


In [3]:
### CHO
CHO_data = data.drop(['TruncatedID','Protein','Carb'], axis=1)
print(CHO_data.Fat.unique())
print(CHO_data.ID.unique())


[20.  18.   2.5]
['2133-002' '2133-022' '2133-020' '2133-004' '2133-010' '2133-015'
 '2133-040' '2133-013' '2133-008' '2133-019' '2133-001' '2133-024'
 '2133-011' '2133-018' '2133-006' '2133-032' '2133-025' '2133-017'
 '2133-028' '2133-039' '2133-021' '2133-030' '2133-033' '2133-037'
 '2133-026' '2133-012' '2133-036' '2133-035' '2133-041' '2133-009']


In [4]:
no_of_trees = [int(x) for x in np.linspace(start=10, stop= 100, num = 10)]
max_depth = [int(x) for x in np.linspace(1,5,num=3)]
# print(no_of_trees,max_depth)
combination = [[x,y] for x in no_of_trees for y in max_depth]
iiter = len(combination)
# print(iiter)

In [5]:
###for LOW-HIGH

CHO_LH_data = CHO_data[CHO_data['Fat'].isin([2.5,20.0])]
CHO_LH_data.loc[CHO_LH_data.Fat == 2.5, 'CHO_bin'] = 0
CHO_LH_data.loc[CHO_LH_data.Fat== 20.0, 'CHO_bin'] = 1
CHO_LH_data = CHO_LH_data.drop(['Fat'], axis=1)

X = CHO_LH_data.loc[:, CHO_LH_data.columns != 'CHO_bin'] #Feature set
Y = CHO_LH_data.loc[:, CHO_LH_data.columns == 'CHO_bin'] #target value/label

values =['2133-002', '2133-022', '2133-020', '2133-004', '2133-010', '2133-015',
 '2133-040' ,'2133-013' ,'2133-008' ,'2133-019' ,'2133-001', '2133-024',
 '2133-011' ,'2133-018', '2133-006', '2133-032' ,'2133-025', '2133-017',
 '2133-028' ,'2133-039', '2133-021', '2133-030', '2133-033', '2133-037',
 '2133-026', '2133-012' ,'2133-036', '2133-035', '2133-041' ,'2133-009'] #needed for manual loso
final_accuracy =[] #will be used for finding mean across nested cross validation
final_accuracy_selected =[]
for i in range(0,30):
    print("For fold :", i+1)
    test =CHO_LH_data[CHO_LH_data['ID'].isin([values[i]])]
    train =CHO_LH_data[~CHO_LH_data['ID'].isin([values[i]])]

    max_max_acc = 0
    arr = np.empty((8, iiter), dtype=object)
    index_mat = np.empty((8,1),dtype=object)
    for fe in range(1,9):
        X_train = train.loc[:, train.columns != 'CHO_bin']
        Y_train = train.loc[:, train.columns == 'CHO_bin']
        X_test = test.loc[:, test.columns != 'CHO_bin']
    
        Y_test =test.loc[:, test.columns == 'CHO_bin']
        train_ids = X_train['ID']
  
        train_ids = np.array(train_ids)
        train_ids = np.unique(train_ids)
        X_train = X_train.drop(['ID'],axis = 1)
        X_test = X_test.drop(['ID'],axis = 1)
        est = RandomForestClassifier()
        rfe = RFE(est,fe)
        rfe = rfe.fit(X_train, Y_train.values.ravel())
        index_ = np.where(rfe.ranking_!=1)
        index_names = X_train.columns[index_]
        X_train= X_train.drop(index_names,axis=1)
        X_test= X_test.drop(index_names,axis=1)
        max_acc = 0
        for k in range(0,iiter):
            ypred_all = []
            
            for j in range(0,29):
                inner_test = train[train['ID'].isin([train_ids[j]])]
                inner_train = train[~train['ID'].isin([train_ids[j]])]
                xtr = inner_train.loc[:, inner_train.columns != 'CHO_bin']
                ytr = inner_train.loc[:, inner_train.columns == 'CHO_bin']
                xvl = inner_test.loc[:, inner_test.columns != 'CHO_bin']
                yvl = inner_test.loc[:, inner_test.columns == 'CHO_bin']
                xtr = xtr.drop(['ID'],axis = 1)
                xvl = xvl.drop(['ID'],axis = 1)
                xtr= xtr.drop(index_names,axis=1)
                xvl= xvl.drop(index_names,axis=1)  
                
                rf = RandomForestClassifier(n_estimators=combination[k][0], max_depth= combination[k][1])
                rf = rf.fit(xtr,ytr)
                ypred = rf.predict(xvl)
                ypred_all.append(accuracy_score(yvl,ypred))
            ypred_mean = np.mean(ypred_all)
            arr[fe-1,k-1]=ypred_mean

        index_mat[fe-1,0]= index_

    found = np.where(arr == np.max(arr))

    final_index = index_mat[found[0][0]][0]
    bestest_param = found[1][0]
    X_train = train.loc[:, train.columns != 'CHO_bin']
    Y_train = train.loc[:, train.columns == 'CHO_bin']
    X_test = test.loc[:, test.columns != 'CHO_bin']
    
    Y_test =test.loc[:, test.columns == 'CHO_bin']
    train_ids = X_train['ID']
  
    train_ids = np.array(train_ids)
    train_ids = np.unique(train_ids)

    X_train = X_train.drop(['ID'],axis = 1)
    X_test = X_test.drop(['ID'],axis = 1)
    index_names = X_train.columns[final_index]
    X_train_selected= X_train.drop(index_names,axis=1)
    X_test_selected= X_test.drop(index_names,axis=1)
    print("List of features for this fold:",X_train_selected.columns)
#     print(X_test)
#     print("*"*40)
#     print(X_test_selected)
    rf = RandomForestClassifier(n_estimators=combination[bestest_param][0], max_depth= combination[bestest_param][1])
    rf = rf.fit(X_train,Y_train)
    Y_pred = rf.predict(X_test)
    rf= rf.fit(X_train_selected, Y_train)
    Y_pred_selected = rf.predict(X_test_selected)                           
    final_accuracy_selected.append(accuracy_score(Y_test,Y_pred_selected))
    final_accuracy.append(accuracy_score(Y_test,Y_pred))

print("Final Accuracy without feature selection : ", np.mean(final_accuracy))
print("Final Accuracy with feature selection : ", np.mean(final_accuracy_selected))
            
    

For fold : 1
List of features for this fold: Index(['Mean', 'SD', 'AUC', 'Peak', 'Halfpk', 'Skew', 'HHMean'], dtype='object')
For fold : 2
List of features for this fold: Index(['SD', 'Peak', 'Skew', 'HHMean'], dtype='object')
For fold : 3
List of features for this fold: Index(['Mean', 'SD', 'AUC', 'Peak', 'Halfpk', 'Skew', 'HHMean'], dtype='object')
For fold : 4
List of features for this fold: Index(['Mean', 'SD', 'Peak', 'Halfpk', 'Skew', 'HHMean'], dtype='object')
For fold : 5
List of features for this fold: Index(['SD', 'Peak', 'Skew', 'HHMean'], dtype='object')
For fold : 6
List of features for this fold: Index(['SD', 'MaxIndex', 'Skew'], dtype='object')
For fold : 7
List of features for this fold: Index(['Mean', 'SD', 'Peak', 'Halfpk', 'Skew', 'HHMean'], dtype='object')
For fold : 8
List of features for this fold: Index(['SD', 'Skew', 'HHMean'], dtype='object')
For fold : 9
List of features for this fold: Index(['Mean', 'SD', 'AUC', 'Peak', 'Skew', 'HHMean'], dtype='object')
For 

In [6]:
###for MEDIUM-HIGH

CHO_LH_data = CHO_data[CHO_data['Fat'].isin([18.0,20.0])]
CHO_LH_data.loc[CHO_LH_data.Fat == 18.0, 'CHO_bin'] = 0
CHO_LH_data.loc[CHO_LH_data.Fat== 20.0, 'CHO_bin'] = 1
CHO_LH_data = CHO_LH_data.drop(['Fat'], axis=1)

X = CHO_LH_data.loc[:, CHO_LH_data.columns != 'CHO_bin'] #Feature set
Y = CHO_LH_data.loc[:, CHO_LH_data.columns == 'CHO_bin'] #target value/label

values =['2133-002', '2133-022', '2133-020', '2133-004', '2133-010', '2133-015',
 '2133-040' ,'2133-013' ,'2133-008' ,'2133-019' ,'2133-001', '2133-024',
 '2133-011' ,'2133-018', '2133-006', '2133-032' ,'2133-025', '2133-017',
 '2133-028' ,'2133-039', '2133-021', '2133-030', '2133-033', '2133-037',
 '2133-026', '2133-012' ,'2133-036', '2133-035', '2133-041' ,'2133-009'] #needed for manual loso
final_accuracy =[] #will be used for finding mean across nested cross validation
final_accuracy_selected =[]
for i in range(0,30):
    print("For fold :", i+1)
    test =CHO_LH_data[CHO_LH_data['ID'].isin([values[i]])]
    train =CHO_LH_data[~CHO_LH_data['ID'].isin([values[i]])]

    max_max_acc = 0
    arr = np.empty((8, iiter), dtype=object)
    index_mat = np.empty((8,1),dtype=object)
    for fe in range(1,9):
        X_train = train.loc[:, train.columns != 'CHO_bin']
        Y_train = train.loc[:, train.columns == 'CHO_bin']
        X_test = test.loc[:, test.columns != 'CHO_bin']
    
        Y_test =test.loc[:, test.columns == 'CHO_bin']
        train_ids = X_train['ID']
  
        train_ids = np.array(train_ids)
        train_ids = np.unique(train_ids)
        X_train = X_train.drop(['ID'],axis = 1)
        X_test = X_test.drop(['ID'],axis = 1)
        est = RandomForestClassifier()
        rfe = RFE(est,fe)
        rfe = rfe.fit(X_train, Y_train.values.ravel())
        index_ = np.where(rfe.ranking_!=1)
        index_names = X_train.columns[index_]
        X_train= X_train.drop(index_names,axis=1)
        X_test= X_test.drop(index_names,axis=1)
        max_acc = 0
        for k in range(0,iiter):
            ypred_all = []
            
            for j in range(0,29):
                inner_test = train[train['ID'].isin([train_ids[j]])]
                inner_train = train[~train['ID'].isin([train_ids[j]])]
                xtr = inner_train.loc[:, inner_train.columns != 'CHO_bin']
                ytr = inner_train.loc[:, inner_train.columns == 'CHO_bin']
                xvl = inner_test.loc[:, inner_test.columns != 'CHO_bin']
                yvl = inner_test.loc[:, inner_test.columns == 'CHO_bin']
                xtr = xtr.drop(['ID'],axis = 1)
                xvl = xvl.drop(['ID'],axis = 1)
                xtr= xtr.drop(index_names,axis=1)
                xvl= xvl.drop(index_names,axis=1)  
                
                rf = RandomForestClassifier(n_estimators=combination[k][0], max_depth= combination[k][1])
                rf = rf.fit(xtr,ytr)
                ypred = rf.predict(xvl)
                ypred_all.append(accuracy_score(yvl,ypred))
            ypred_mean = np.mean(ypred_all)
            arr[fe-1,k-1]=ypred_mean

        index_mat[fe-1,0]= index_

    found = np.where(arr == np.max(arr))

    final_index = index_mat[found[0][0]][0]
    bestest_param = found[1][0]
    X_train = train.loc[:, train.columns != 'CHO_bin']
    Y_train = train.loc[:, train.columns == 'CHO_bin']
    X_test = test.loc[:, test.columns != 'CHO_bin']
    
    Y_test =test.loc[:, test.columns == 'CHO_bin']
    train_ids = X_train['ID']
  
    train_ids = np.array(train_ids)
    train_ids = np.unique(train_ids)

    X_train = X_train.drop(['ID'],axis = 1)
    X_test = X_test.drop(['ID'],axis = 1)
    index_names = X_train.columns[final_index]
    X_train_selected= X_train.drop(index_names,axis=1)
    X_test_selected= X_test.drop(index_names,axis=1)
    print("List of features for this fold:",X_train_selected.columns)
#     print(X_test)
#     print("*"*40)
#     print(X_test_selected)
    rf = RandomForestClassifier(n_estimators=combination[bestest_param][0], max_depth= combination[bestest_param][1])
    rf = rf.fit(X_train,Y_train)
    Y_pred = rf.predict(X_test)
    rf= rf.fit(X_train_selected, Y_train)
    Y_pred_selected = rf.predict(X_test_selected)                           
    final_accuracy_selected.append(accuracy_score(Y_test,Y_pred_selected))
    final_accuracy.append(accuracy_score(Y_test,Y_pred))

print("Final Accuracy without feature selection : ", np.mean(final_accuracy))
print("Final Accuracy with feature selection : ", np.mean(final_accuracy_selected))
            
    

For fold : 1
List of features for this fold: Index(['Mean', 'SD', 'Peak', 'Skew'], dtype='object')
For fold : 2
List of features for this fold: Index(['Peak', 'HHMean'], dtype='object')
For fold : 3
List of features for this fold: Index(['SD', 'Peak', 'Skew', 'HHMean'], dtype='object')
For fold : 4
List of features for this fold: Index(['Mean', 'SD', 'Peak', 'MaxIndex', 'Skew'], dtype='object')
For fold : 5
List of features for this fold: Index(['Peak', 'Skew'], dtype='object')
For fold : 6
List of features for this fold: Index(['Peak', 'Skew'], dtype='object')
For fold : 7
List of features for this fold: Index(['SD', 'Peak'], dtype='object')
For fold : 8
List of features for this fold: Index(['SD', 'Peak'], dtype='object')
For fold : 9
List of features for this fold: Index(['Peak'], dtype='object')
For fold : 10
List of features for this fold: Index(['Peak'], dtype='object')
For fold : 11
List of features for this fold: Index(['Peak', 'Skew'], dtype='object')
For fold : 12
List of fea

In [7]:
###for LOW-Medium

CHO_LH_data = CHO_data[CHO_data['Fat'].isin([2.5,18.0])]
CHO_LH_data.loc[CHO_LH_data.Fat == 2.5, 'CHO_bin'] = 0
CHO_LH_data.loc[CHO_LH_data.Fat== 18.0, 'CHO_bin'] = 1
CHO_LH_data = CHO_LH_data.drop(['Fat'], axis=1)

X = CHO_LH_data.loc[:, CHO_LH_data.columns != 'CHO_bin'] #Feature set
Y = CHO_LH_data.loc[:, CHO_LH_data.columns == 'CHO_bin'] #target value/label

values =['2133-002', '2133-022', '2133-020', '2133-004', '2133-010', '2133-015',
 '2133-040' ,'2133-013' ,'2133-008' ,'2133-019' ,'2133-001', '2133-024',
 '2133-011' ,'2133-018', '2133-006', '2133-032' ,'2133-025', '2133-017',
 '2133-028' ,'2133-039', '2133-021', '2133-030', '2133-033', '2133-037',
 '2133-026', '2133-012' ,'2133-036', '2133-035', '2133-041' ,'2133-009'] #needed for manual loso
final_accuracy =[] #will be used for finding mean across nested cross validation
final_accuracy_selected =[]
for i in range(0,30):
    print("For fold :", i+1)
    test =CHO_LH_data[CHO_LH_data['ID'].isin([values[i]])]
    train =CHO_LH_data[~CHO_LH_data['ID'].isin([values[i]])]

    max_max_acc = 0
    arr = np.empty((8, iiter), dtype=object)
    index_mat = np.empty((8,1),dtype=object)
    for fe in range(1,9):
        X_train = train.loc[:, train.columns != 'CHO_bin']
        Y_train = train.loc[:, train.columns == 'CHO_bin']
        X_test = test.loc[:, test.columns != 'CHO_bin']
    
        Y_test =test.loc[:, test.columns == 'CHO_bin']
        train_ids = X_train['ID']
  
        train_ids = np.array(train_ids)
        train_ids = np.unique(train_ids)
        X_train = X_train.drop(['ID'],axis = 1)
        X_test = X_test.drop(['ID'],axis = 1)
        est = RandomForestClassifier()
        rfe = RFE(est,fe)
        rfe = rfe.fit(X_train, Y_train.values.ravel())
        index_ = np.where(rfe.ranking_!=1)
        index_names = X_train.columns[index_]
        X_train= X_train.drop(index_names,axis=1)
        X_test= X_test.drop(index_names,axis=1)
        max_acc = 0
        for k in range(0,iiter):
            ypred_all = []
            
            for j in range(0,29):
                inner_test = train[train['ID'].isin([train_ids[j]])]
                inner_train = train[~train['ID'].isin([train_ids[j]])]
                xtr = inner_train.loc[:, inner_train.columns != 'CHO_bin']
                ytr = inner_train.loc[:, inner_train.columns == 'CHO_bin']
                xvl = inner_test.loc[:, inner_test.columns != 'CHO_bin']
                yvl = inner_test.loc[:, inner_test.columns == 'CHO_bin']
                xtr = xtr.drop(['ID'],axis = 1)
                xvl = xvl.drop(['ID'],axis = 1)
                xtr= xtr.drop(index_names,axis=1)
                xvl= xvl.drop(index_names,axis=1)  
                
                rf = RandomForestClassifier(n_estimators=combination[k][0], max_depth= combination[k][1])
                rf = rf.fit(xtr,ytr)
                ypred = rf.predict(xvl)
                ypred_all.append(accuracy_score(yvl,ypred))
            ypred_mean = np.mean(ypred_all)
            arr[fe-1,k-1]=ypred_mean

        index_mat[fe-1,0]= index_

    found = np.where(arr == np.max(arr))

    final_index = index_mat[found[0][0]][0]
    bestest_param = found[1][0]
    X_train = train.loc[:, train.columns != 'CHO_bin']
    Y_train = train.loc[:, train.columns == 'CHO_bin']
    X_test = test.loc[:, test.columns != 'CHO_bin']
    
    Y_test =test.loc[:, test.columns == 'CHO_bin']
    train_ids = X_train['ID']
  
    train_ids = np.array(train_ids)
    train_ids = np.unique(train_ids)

    X_train = X_train.drop(['ID'],axis = 1)
    X_test = X_test.drop(['ID'],axis = 1)
    index_names = X_train.columns[final_index]
    X_train_selected= X_train.drop(index_names,axis=1)
    X_test_selected= X_test.drop(index_names,axis=1)
    print("List of features for this fold:",X_train_selected.columns)
#     print(X_test)
#     print("*"*40)
#     print(X_test_selected)
    rf = RandomForestClassifier(n_estimators=combination[bestest_param][0], max_depth= combination[bestest_param][1])
    rf = rf.fit(X_train,Y_train)
    Y_pred = rf.predict(X_test)
    rf= rf.fit(X_train_selected, Y_train)
    Y_pred_selected = rf.predict(X_test_selected)                           
    final_accuracy_selected.append(accuracy_score(Y_test,Y_pred_selected))
    final_accuracy.append(accuracy_score(Y_test,Y_pred))

print("Final Accuracy without feature selection : ", np.mean(final_accuracy))
print("Final Accuracy with feature selection : ", np.mean(final_accuracy_selected))
            
    

For fold : 1
List of features for this fold: Index(['Mean', 'SD', 'AUC', 'Peak', 'MaxIndex', 'Halfpk', 'Skew', 'HHMean'], dtype='object')
For fold : 2
List of features for this fold: Index(['SD'], dtype='object')
For fold : 3
List of features for this fold: Index(['Mean', 'SD', 'AUC', 'Peak', 'MaxIndex', 'Halfpk', 'Skew', 'HHMean'], dtype='object')
For fold : 4
List of features for this fold: Index(['Mean', 'SD', 'AUC', 'Peak', 'Halfpk', 'Skew', 'HHMean'], dtype='object')
For fold : 5
List of features for this fold: Index(['Mean', 'SD', 'AUC', 'Skew', 'HHMean'], dtype='object')
For fold : 6
List of features for this fold: Index(['Mean', 'SD', 'AUC'], dtype='object')
For fold : 7
List of features for this fold: Index(['Mean', 'SD'], dtype='object')
For fold : 8
List of features for this fold: Index(['SD'], dtype='object')
For fold : 9
List of features for this fold: Index(['SD'], dtype='object')
For fold : 10
List of features for this fold: Index(['Mean', 'SD', 'AUC', 'Peak', 'MaxIndex'